In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/data/tim/heronWorkspace/src")
sys.path.append("/data/tim/heronWorkspace/AE")
sys.path.append("/data/tim/heronWorkspace/classifier")
sys.path.append("/data/tim/heronWorkspace/")


from AEHeronModelV1 import AEHeronModel
from AEHeronModelV2 import CAEHeron
from lightning.pytorch.callbacks import ModelSummary
from torchsummary import summary
import HeronImageLoader
from torch.utils.data import DataLoader, BatchSampler
from matplotlib import pyplot as plt
import lightning.pytorch as pl
from lightning.pytorch.tuner import Tuner
import pandas as pd
from lightning.pytorch.loggers import CSVLogger
from MLPV1 import MLP, MLPMSEHeatMap
from models import MLPBasic, CAEBigBottleneck, CAESmallBottleneckWithLinear, MLPBasicHeatMap, CAEV1
import numpy as np
import torch.nn.functional as F
import torch
from torchvision.transforms import GaussianBlur
from PIL import Image, ImageFilter
import random
from scipy.stats import loguniform
from ClassifierDatasets import DatasetThreeConsecutive, UnNormalize
# from torchmetrics.image import StructuralSimilarityIndexMeasure
from skimage.metrics import structural_similarity as ssim
import seaborn as sns
from sklearn.model_selection import ParameterSampler
from scipy.stats import loguniform
import functorch
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
# from torchmetrics import ConfusionMatrix

sns.set_theme()




# Params:
- different CAE
- cameraProps - cutoff on the sides
- gaussian filter params / min-filter params
- zero thresholds
- sum threshold
- dataset props

In [15]:
loaderParams = dict(
    lblValidationMode = "Manual",
    balanced = True,
    anomalyObviousness = "obvious",
    distinctCAETraining = False,
    colorMode = "RGB",
    random_state = 1,
    set = "all"
)

distributions = dict(
    cameras = [["NEN1", "SBU3"]], #, ["SBU4"]
    balanced = [True, False],
    distinctCAETraining = [True, False],
    gaussianFilterSize = [5],
    gaussianFilterSigma = [3, 5],
    filter = ["MinFilter"], #["MinFilter", "GaussianFilter"]
    zeroThreshold = np.random.uniform(low=0.15, high=0.4, size=30), # uniform dist on loc, loc+scale -> uniform(loc, scale) #threshold for zeroing out the image
    sumThreshold = np.random.uniform(low=20, high=50, size=30),
    lossFn = ["L1"]#["MSE", "L1"],
)

sampler = ParameterSampler(distributions, n_iter=3, random_state=loaderParams["random_state"])

includeCameraProps = False
model = CAEV1

checkPoint = '/data/tim/heronWorkspace/logs/CAEV1/version_3/checkpoints/epoch=49-step=19350.ckpt'
caeLoaded = CAEHeron.load_from_checkpoint(checkPoint, model = model)
caeLoaded.freeze()
print(caeLoaded.hparams)

fileName = "basicCAETest"
columns = ["trueLbl", "predictedLbl", "ImagePath", "includeCameraProps", "CAECheckPoint", "isCAETrainingCamera"] + list(loaderParams.keys()) + list(distributions.keys()) 
try:
    df = pd.read_csv(f"/data/tim/heronWorkspace/caeTestData/{fileName}.csv")
except:
    df = pd.DataFrame(columns=columns)


"batch_size":         16
"bottleneck":         128
"cameras":            ['NEN1', 'SBU3']
"gammaScheduler":     0.7879353477950641
"ldim":               16
"learning_rate":      0.003533238255084643
"num_workers_loader": 4
"transforms":         None
"weight_decay":       4.5387898639621974e-07


In [6]:
def min_filter(tensor : torch.Tensor, kernel_size=3):
    # Unfold the tensor into sliding local blocks
    unfolded = tensor.unfold(0, kernel_size, 1)
    unfolded = unfolded.unfold(1, kernel_size, 1)
    # Compute the minimum in each of these blocks
    return unfolded.min(dim=-1)[0].min(dim=-1)[0]

In [7]:
a = torch.tensor([[[0] * 5, [1] * 5, [2] * 5, [3] * 5, [4] * 5]]).unfold(1, 3, 1)
a.unfold(2, 3, 1)

a = torch.tensor([[0] * 5, [1] * 5, [2] * 5, [3] * 5, [4] * 5]).unfold(0, 3, 1)
a.unfold(1, 3, 1).shape

torch.Size([3, 3, 3, 3])

try: unnorm only on result

In [19]:
#TODO: include camerProps

# confMat = ConfusionMatrix(task="binary", num_classes=2)
unnorm = UnNormalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))

negVals = {
        "mean": [],
        "std": [],
        "min": [],
        "max": []
    }
posVals = {
    "mean": [],
    "std": [],
    "min": [],
    "max": []
}

for params in sampler:
    dataset = DatasetThreeConsecutive(cameras=params["cameras"], resize_to=CAEV1.imsize, **loaderParams)
    print(f'Length of dataset: {len(dataset)}')
    print(params)
    dataLoader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=2)

    blur = GaussianBlur(kernel_size=5, sigma=3) #TODO: make this a parameter
    lossFn = F.mse_loss if params["lossFn"] == "MSE" else F.l1_loss
    vals = []

    for (imArr, lblArr, camera, ImagePath) in dataLoader:
        isTrainingCamera = camera in caeLoaded.hparams.cameras
        prevImg = imArr[0] #alwasy #batch_size images
        currImg = imArr[1]
        nextImg = imArr[2]

        prevPred, currPred, nextPred = [unnorm(caeLoaded(x.to(caeLoaded.device))) for x in [prevImg, currImg, nextImg]]
        prevImg, currImg, nextImg = [unnorm(x) for x in [prevImg, currImg, nextImg]]

        # plt.imshow(currImg[0].cpu().detach().numpy().transpose(1, 2, 0))
        # plt.show()

        prevImgBlurred, currImgBlurred, nextImgBlurred = [blur.forward(x).to(prevPred.device) for x in [prevImg, currImg, nextImg]]
        # plt.imshow(currImgBlurred[0].cpu().detach().numpy().transpose(1, 2, 0))
        # plt.show()
        
        prevImd, currImd, nextImd = [torch.sum(lossFn(imgBlurred, pred, reduction='none'), dim=1) for imgBlurred, pred in zip([prevImgBlurred, currImgBlurred, nextImgBlurred], [prevPred, currPred, nextPred])]

        prevToCurrImd = torch.clamp(torch.sub(currImd, prevImd), min= 0)
        nextToCurrImd = torch.clamp(torch.sub(currImd, nextImd), min= 0)

        prevNextCurrImd = torch.div(torch.add(prevToCurrImd, nextToCurrImd), 2)

        prevNextCurrImdMin = torch.stack([min_filter(x, kernel_size=3) for x in prevNextCurrImd]) #TODO: evtl make this as before
        

        # plt.imshow(prevNextCurrImdMin[0].cpu().detach().numpy(), cmap="hot")
        # print(torch.sum(prevNextCurrImdMin[0]).item())
        # plt.show()
        # plt.imshow(prevNextCurrImdMin[1].cpu().detach().numpy(), cmap="hot")
        # print(torch.sum(prevNextCurrImdMin[1]).item())
        # plt.show()

        # prevNextCurrImdMin = torch.div(prevNextCurrImdMin, 255)
        # plt.imshow(prevNextCurrImdMin[0].cpu().detach().numpy(), cmap="hot")
        # plt.show()

        # print(torch.max(prevNextCurrImdMin))
        prevNextCurrImdMinThresh = torch.where(prevNextCurrImdMin < params["zeroThreshold"], torch.zeros_like(prevNextCurrImdMin), prevNextCurrImdMin)
        # plt.imshow(prevNextCurrImdMinThresh[0].cpu().detach().numpy(), cmap="hot")
        # plt.show()

        sumPrevNextCurrImdMin = torch.sum(prevNextCurrImdMinThresh, dim=(1, 2))
        predictions = (sumPrevNextCurrImdMin> params["sumThreshold"]).to(torch.int)


        for i in range(len(predictions)):
            vals.append([lblArr[i].item(), predictions[i].item(), sumPrevNextCurrImdMin[i].item()])
        #     dfNew = pd.DataFrame(columns=columns, )
        #     dfNew = df.append(dict(zip(columns, [lblArr[i].item(), predictions[i].item(), ImagePath[i], includeCameraProps, checkPoint, isTrainingCamera] + list(loaderParams.values()) + list(params.values()))), ignore_index=True)
        
        # df.to_csv(f"/data/tim/heronWorkspace/caeTestData/{fileName}.csv", index=False)

        # print(predictions)
        # print(prevNextCurrImdMin.shape)
        # print(torch.max(prevNextCurrImdMin))
        # print(imArr[0].shape)
        # print(len(imArr))
        # print(len(lblArr))

    # confMat
    vals = np.array(vals)
    # print(vals)
    # print(vals[:, 0])
    # print(vals[:, 1])
    confMat = confusion_matrix(vals[:, 0], vals[:,1])
    fig= plt.figure(figsize = (6,3))
    ax = fig.add_subplot(111)
    sns.heatmap(confMat, annot=True)
    ax.set_xlabel('Pred', fontsize=12)
    ax.set_ylabel('True', fontsize=12)
    onlyPos = vals[vals[:, 0] == 1]
    onlyNeg = vals[vals[:, 0] == 0]
    print(f'mean pos: {np.mean(onlyPos[:, 2])}, std pos: {np.std(onlyPos[:, 2])}')
    print(f'mean neg: {np.mean(onlyNeg[:, 2])}, std neg: {np.std(onlyNeg[:, 2])}')
    print(f'lowest pos: {np.min(onlyPos[:, 2])}, highest pos: {np.max(onlyPos[:, 2])}')
    print(f'lowest neg: {np.min(onlyNeg[:, 2])}, highest neg: {np.max(onlyNeg[:, 2])}')

    posVals["mean"].append(np.mean(onlyPos[:, 2]))
    posVals["std"].append(np.std(onlyPos[:, 2]))
    posVals["min"].append(np.min(onlyPos[:, 2]))
    posVals["max"].append(np.max(onlyPos[:, 2]))

    negVals["mean"].append(np.mean(onlyNeg[:, 2]))
    negVals["std"].append(np.std(onlyNeg[:, 2]))
    negVals["min"].append(np.min(onlyNeg[:, 2]))
    negVals["max"].append(np.max(onlyNeg[:, 2]))

    print(f'accuracy: {accuracy_score(vals[:, 0], vals[:,1])}')
    print(f'precision: {precision_score(vals[:, 0], vals[:,1])}')
    print(f'recall: {recall_score(vals[:, 0], vals[:,1])}')
    print(f'f1: {f1_score(vals[:, 0], vals[:,1])}')
    plt.show()
    vals = []

        
fig = plt.figure(figsize = (6,3))
plt.errorbar(range(len(posVals["mean"])), posVals["mean"], yerr=posVals["std"], fmt='o', label="pos", color="blue")
plt.errorbar(range(len(negVals["mean"])), negVals["mean"], yerr=negVals["std"], fmt='-', label="neg", color="red")
plt.plot(range(len(posVals["mean"])), posVals["max"], 'o', color="blue")
plt.plot(range(len(negVals["mean"])), negVals["max"], '-', color="red")
plt.plot(range(len(posVals["mean"])), posVals["min"], 'o', color="blue")
plt.plot(range(len(negVals["mean"])), negVals["min"], '-', color="red")
plt.xlabel("iteration")
plt.xlim([-1, 5])
plt.legend()
plt.show()

# confusionMatrix
# confMat = confusion_matrix(df["trueLbl"], df["predictedLbl"])
# plt.figure(figsize = (6,3))
# sns.heatmap(confMat, annot=True)
# plt.show()

Length of dataset: 62
{'zeroThreshold': 0.2455403529275042, 'sumThreshold': 27.85840753027775, 'lossFn': 'L1', 'gaussianFilterSize': 5, 'gaussianFilterSigma': 5, 'filter': 'MinFilter', 'distinctCAETraining': True, 'cameras': ['NEN1', 'SBU3'], 'balanced': False}


In [18]:
range(len(posVals["mean"]))

range(0, 1)

In [ ]:
try:
    df = pd.read_csv("./resultsBasicClassifier.csv")
except:
    df = pd.DataFrame(columns = params.keys(), )